## Paso 0: Importar librerías

Antes de empezar, cargamos las librerías necesarias para manejar datos, hacer gráficos, preparar nuestros datos y entrenar modelos. Esto nos permite tener todas las herramientas listas desde el principio.

In [ ]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y preprocesamiento
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Modelos de ejemplo
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Métricas
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score

# Otros
import warnings
warnings.filterwarnings('ignore')  # evita mensajes molestos en el notebook

pd.options.mode.copy_on_write = True

## Paso 1: Cargar datos

Primero necesitamos traer nuestros datos al entorno de trabajo. Usualmente vienen en archivos CSV, Excel o desde una base de datos. En esta etapa solo leemos los datos y echamos un primer vistazo rápido.

In [ ]:
# Cargar un CSV
df = pd.read_csv("data/ejemplo_housing.csv")

# Primeras filas para ver cómo es el dataset
df.head()

# Información básica del dataset
df.info()

# Estadísticas básicas de las columnas numéricas
df.describe()


## Paso 2: Definición del problema

Antes de tocar los datos, debemos saber qué queremos resolver:

¿Queremos predecir un valor numérico? → Regresión

¿Queremos clasificar en categorías? → Clasificación

Esto nos ayuda a decidir qué modelos y métricas usar.

Advertencia: A veces los targets son números pero representan categorías (por ejemplo, códigos de producto o clases), en ese caso es clasificación, no regresión.

## Paso 3: Divisón de Train y Test



Antes de empezar a explorar o modelar los datos, siempre separamos una parte para probar nuestro modelo después:

Train → para entrenar el modelo

Test → para evaluar si el modelo generaliza bien



Se suele usar 70-80% para train y 20-30% para test

## Paso 4: Limpieza de los datos


Antes de explorar los datos, conviene hacer una limpieza básica: revisar valores faltantes graves, asegurarse de que los tipos son correctos y tratar codificaciones simples.

La estandarización y transformaciones más avanzadas se harán después, en el tratamiento de variables.

In [ ]:
# Revisar valores faltantes
print(df.isna().sum())

# Rellenar NaN simples (ejemplo con media)
df['edad'] = df['edad'].fillna(df['edad'].mean())

# Eliminar filas duplicadas si es necesario
df = df.drop_duplicates()

# Revisar tipos de datos
print(df.dtypes)

## Paso 5: Comprensión de variables, mini-EDA.

### Pairplot inicial

Visualización rápida de varias variables a la vez. Útil para detectar patrones, clusters o interacciones importantes, especialmente cuando no hay demasiadas features continuas.


### Estudio de la target

Antes de mirar cualquier feature, observa tu variable objetivo. Comprueba su distribución y si está desbalanceada: esto influye en la elección de métricas y modelos. Para clasificación, revisa cuántas muestras hay de cada clase; para regresión, analiza la dispersión y posibles outliers.

In [ ]:
# Clasificación
sns.countplot(x='target', data=df)

# Regresión
sns.histplot(df['target'])

### Análisis de correlaciones

Revisa cómo se relacionan tus features entre sí y con el target. Correlaciones muy altas pueden indicar redundancia; correlaciones con el target ayudan a ver qué features podrían ser más relevantes.


In [ ]:
# Matriz de correlaciones
sns.heatmap(df.corr(numeric_only = True), annot=True, vmin = -1, vmax = 1, cmap='coolwarm')

Ponemos los parámetros de vmin y vmax en -1 y 1 para ver la distribución estandarizada de las correlaciones.

Cuanto más cerca del 1, será correlación positiva; cuanto más cerca del -1, correlación negativa.

Cuanto más cerca del 0, más irrelevante.

El parámetro de cmap='coolwarm' nos ayuda a analizar esto a golpe de vista.

#### Advertencia sobre multicolinealidad:
Si dos o más features están muy correlacionadas entre sí, decimos que existe multicolinealidad. Esto puede afectar negativamente a algunos modelos (como regresión lineal) porque dificulta que el modelo separe el efecto de cada variable. En ese caso, conviene eliminar o combinar features muy correlacionadas antes de entrenar.

### Análisis univariante de features

Observa cada feature individualmente: distribución, valores extremos, posibles outliers. Esto te ayuda a detectar problemas antes de modelar.

In [ ]:
# Ejemplo: Histograma para todas las features numéricas
df.hist(figsize=(12,8), bins=20)

### Análisis bivariante


Compara cada feature con el target o entre sí para ver relaciones. Para regresión, scatterplots; para clasificación, boxplots o violinplots.

In [ ]:
# Scatterplot para regresión
sns.scatterplot(x='feature1', y='target', data=df)

# Boxplot para clasificación
sns.boxplot(x='target', y='feature2', data=df)

## Paso 6: Tratamiento de variables

En este paso preparamos los datos para que puedan ser usados por modelos de Machine Learning. Incluye la eliminación de información poco útil, el tratamiento de problemas comunes en los datos y la creación o transformación de variables.

### 6.1 Eliminación de features
Tras el EDA, evaluamos si sobran variables. No todas las features aportan información predictiva y algunas pueden incluso perjudicar al modelo.
Suelen eliminarse:
Features con valor constante.
Features con muchos missings (≈ >20–30%).
Identificadores (IDs, nombres propios).
Strings largos sin tratamiento NLP.
Variables con alta cardinalidad.
Features muy correlacionadas entre sí (multicolinealidad).
Estas decisiones dependen del número total de features: cuantos más datos tengamos, más margen hay para eliminar.

In [ ]:
# Eliminar columnas manualmente
df.drop(columns=['feature1', 'feature2'])

### 6.2 Duplicados
Los duplicados no suelen aportar información y normalmente se eliminan.
Antes, es importante identificar qué define una fila única (cliente, pedido, cliente + fecha, etc.).

In [ ]:
# Eliminar duplicados completos
df.drop_duplicates()

# Eliminar duplicados según una o varias columnas
df.drop_duplicates(subset=['cliente'], keep='last')

### 6.3 Missings (valores faltantes)
Un missing es un valor ausente (NaN), no un 0 ni un string vacío.

La mayoría de modelos no los admiten, así que deben tratarse.

Opciones principales:

Eliminar filas o columnas (si son pocos).

- Imputar con media, mediana o moda.

- Imputar con un valor concreto.

- Imputación avanzada (KNN, modelos).

- Imputación + flag indicando que había missing.

In [ ]:
# Eliminar filas con missings
df.dropna()

# Imputar con la media
df['feature'] = df['feature'].fillna(df['feature'].mean())

### 6.4 Anomalías y errores
No son outliers, sino datos incorrectos:

- Valores negativos imposibles.
- Fechas mal leídas (formato, año 9999).
- Problemas de encoding en texto.

Suelen tratarse como missings o corregirse manualmente tras revisión.

### 6.5 Outliers
Valores atípicos que se alejan mucho del resto.

Pueden penalizar modelos basados en distancias o gradiente.

Detección:
- Gráficos (boxplot, histograma, scatter).
- Media ± N·desviación estándar.

Tratamiento:
- Eliminarlos.
- No hacer nada (árboles y SVM son robustos).
- Transformaciones (log).
- Binning.
- Imputación.


⚠️ Si eliminas outliers en X_train, elimina las mismas filas en y_train.

### 6.7 Transformaciones
Buscan mejorar la forma de la distribución (asimetría).

Logarítmica (la más común).

Cuadrada / cúbica.

Box-Cox.

Se evalúa con histogramas, skew o test de Shapiro.

### 6.8 Encodings (variables categóricas)

Los modelos no admiten texto, hay que convertirlo a números.

Elección según el tipo de variable:
- Binaria → mapeo 0/1.
- Ordenada → mapeo respetando el orden.
- No ordenada → OneHot / dummies.
- Alta cardinalidad → Hashing.

In [ ]:
# One-hot encoding
pd.get_dummies(df, columns=['city'])

⚠️ Codificar siempre usando train, y aplicar lo mismo a test.

### 6.9 Escalado

Pone todas las variables en la misma escala.

- StandardScaler (media 0, std 1).

- MinMaxScaler (rango 0–1).

Importante para modelos basados en distancias o gradiente.
No afecta a árboles.


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Paso 7: Elección de métricas

Ningún modelo es perfecto: siempre comete errores. Por eso necesitamos una métrica, que nos sirva para:
- Comparar distintos modelos y elegir el mejor.
- Comunicar resultados de forma clara y honesta.

La métrica debe elegirse según el tipo de problema y el contexto, no solo porque sea la más conocida. La mayoría están implementadas en sklearn.

### 7.1 Métricas para clasificación
#### Accuracy
- Porcentaje total de aciertos.
- Fácil de entender.
- Poco informativa si las clases están desbalanceadas.

#### Matriz de confusión
Muestra en qué se equivoca el modelo.
- TP: positivos bien clasificados
- TN: negativos bien clasificados
- FP: falsos positivos
- FN: falsos negativos

Es la base para entender el resto de métricas.

#### Recall (Sensibilidad)
De todos los positivos reales, cuántos detecta el modelo.

Importante cuando no queremos FN.

Ejemplo típico: detección de enfermedades.

#### Precision
De todos los positivos predichos, cuántos son correctos.

Importante cuando no queremos FP.

Ejemplo: filtro de spam.

#### Curvas Precision–Recall
Permiten analizar el comportamiento del modelo cambiando el threshold.
- Útiles cuando las clases están desbalanceadas.
- Ayudan a elegir el punto óptimo según el problema.

#### Curva ROC y AUC
Relaciona Recall con la tasa de falsos positivos.
- AUC mide el rendimiento global del clasificador.
- AUC ≈ 1 → buen modelo
- AUC ≈ 0.5 → modelo aleatorio
Más usada en datasets balanceados.

#### ¿Cómo saber qué metrica de clasificación usar?
Imagina un modelo que detecta si un email es spam. Si solo miramos accuracy, puede parecer muy bueno porque acierta casi todo, pero quizá falla justo en los correos importantes. Recall sería clave si no queremos que se nos escape ningún spam (aunque marque algún correo bueno como spam), mientras que precision es más importante si nos preocupa no perder ningún correo importante. La matriz de confusión nos permite ver exactamente en qué se equivoca el modelo: qué correos confunde y cómo. Cuando no está claro si priorizar recall o precision, usamos el F1-score, que busca un equilibrio entre ambos. Si además queremos analizar cómo cambia el comportamiento del modelo al ser más o menos estricto, usamos curvas como ROC o precision–recall, que nos ayudan a elegir el punto óptimo según el coste real de cada tipo de error.

### 7.2 Métricas para regresión
#### R² (coeficiente de determinación)
- Mide la proporción de variabilidad explicada por el modelo.
- No indica por sí solo si el modelo es bueno.
- Puede ser engañoso si se usa sin contexto.

#### MSE (Mean Squared Error)
- Media del error al cuadrado.
- Penaliza mucho los errores grandes.
- Útil para comparar modelos, no para explicar resultados.

#### RMSE (Root MSE)
- Raíz del MSE, en las unidades del target.
- Más interpretable que MSE.
- Muy usada para reportar resultados.

#### MAE (Mean Absolute Error)
- Error medio absoluto.
- Fácil de interpretar.
- Más robusta frente a outliers.

#### MAPE (Mean Absolute Percentage Error)
- Error medio en porcentaje.
- Útil para comparar modelos con distintos targets.
- Acotada entre 0 y 100.
- Problemas si el target tiene valores cercanos a 0.

## Paso 8: Selección del modelo

No existe un modelo “mejor” en general: el modelo adecuado depende de los datos y del objetivo del problema. Antes de entrenar, conviene tener en cuenta varios factores clave.

### Volumen de datos y número de features
- Pocos datos y muchas variables: conviene usar modelos simples, con mayor sesgo pero que generalizan mejor (regresión lineal, Naive Bayes, SVM lineal).
- Muchos datos y pocas variables: se pueden usar modelos más complejos, capaces de capturar patrones más finos (KNN, árboles de decisión, SVM con kernel).
### Precisión vs interpretabilidad
- Uno de los compromisos más importantes.
- Modelos interpretables (“caja blanca”) permiten explicar por qué toman una decisión (regresiones, modelos lineales).
- Modelos más complejos (“caja negra”) suelen ser más precisos, pero difíciles de explicar (ensembles, redes neuronales).
- La elección depende de si necesitamos justificar las decisiones del modelo.
### Velocidad de entrenamiento
- Rápidos: regresión logística, modelos lineales, Naive Bayes.
- Lentos: SVM (especialmente con tuning), ensembles, redes neuronales.
- Esto es importante cuando hay muchos datos o poco tiempo de cómputo.
### Tipo de relación entre los datos
- Si la relación con el target es aproximadamente lineal, los modelos lineales suelen funcionar bien.
- Si existen relaciones no lineales, conviene usar modelos más flexibles como árboles, random forest, KNN o redes neuronales.
- Esto se detecta principalmente durante el EDA.

## Paso 9: Entrenamiento del modelo

En este paso el modelo aprende patrones a partir de los datos de entrenamiento. El objetivo no es solo que funcione bien en train, sino que generalice correctamente a datos nuevos.

Antes de entrenar, es importante:
- Haber separado correctamente train y test.
- Aplicar el preprocesado solo con train.
- Definir claramente la métrica con la que se evaluará el modelo.

Durante el entrenamiento:
- Se entrena primero un modelo base (baseline) sin mucho ajuste.
- Después, si es necesario, se prueban otros modelos o se ajustan hiperparámetros.
- El uso de pipelines asegura que el preprocesado y el modelo se apliquen siempre de forma consistente y sin fugas de información.

Tras entrenar:
- Se evalúa el modelo en test, nunca en train.
- Se comparan resultados con otros modelos.
- Se decide si el rendimiento es suficiente o si hay que volver a pasos anteriores (features, modelo o métrica).


In [ ]:
# Crear y entrenar el modelo
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

## Paso 10: Hiperparametrización/Regularización

### Elegir hiperparámetros

Como ya sabes, cada dataset es de su padre y de su madre, y por tanto es imposible determinar el modelo con sus hiperparámetros que mejor se ajusten a los datos. Por tanto, tendremos que probar varias combinaciones. Por suerte sklearn tiene una función llamada GridSearchCV que permite probar varias combinaciones de una manera automatizada.

Empieza iterando unos pocos hiperparámetros y luego ve subiendo, según los resultados de esa ejecución.

En este apartado veremos posibles hiperparámetros a emplear en los algoritmos más utilizados.

In [ ]:
# REGRESION LOGISTICA
grid_logreg = {
                     "penalty": ["l1","l2"], # Regularizaciones L1 y L2.
                     "C": [0.1, 0.5, 1.0, 5.0], # Cuanta regularizacion queremos

                     "max_iter": [50,100,500],  # Iteraciones del Gradient Descent. No suele impactar mucho
                                                # pero en ocasiones aparecen warnings diciendo que se aumente

                     "solver": ["liblinear"]  # Suele ser el más rápido
                    }


# ARBOL DE DECISION
grid_arbol = {
    "max_depth": list(range(1, 10)),
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5]
}

# RANDOM FOREST
grid_random_forest = {"n_estimators": [120], # El Random Forest no suele empeorar por exceso de
                                             # estimadores. A partir de cierto numero no merece la pena
                                             # perder el tiempo ya que no mejora mucho más la precisión.
                                             # Entre 100 y 200 es una buena cifra


                     "max_depth": [3,4,5,6,10,15,17], # No le afecta tanto el overfitting como al decissiontree.
                                                      # Podemos probar mayores profundidades

                     "max_features": ["sqrt", 3, 4] # Numero de features que utiliza en cada split.
                                                    # cuanto más bajo, mejor generalizará y menos overfitting.

                     }


# GRADIENT BOOSTING
grid_gradient_boosting = {"loss": ["log_loss"],
                          "learning_rate": [0.05, 0.1, 0.2, 0.4, 0.5],  # Cuanto más alto, mas aporta cada nuevo arbol

                          "n_estimators": [20,50,100,200], # Cuidado con poner muchos estiamdores ya que vamos a
                                                           # sobreajustar el modelo

                          "max_depth": [1,2,3,4,5], # No es necesario poner una profundiad muy alta. Cada nuevo
                                                    # arbol va corrigiendo el error de los anteriores.


                          "max_features": ["sqrt", 3, 4], # Igual que en el random forest
                          }

Ejemplo de uso completo:

In [ ]:
# Modelo base
model = LogisticRegression(max_iter=1000)

# Grid de hiperparámetros
param_grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l2"]
}

# GridSearch
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

# Entrenamiento SOLO con train
grid.fit(X_train, y_train)

# Mejor modelo encontrado
best_model = grid.best_estimator_

print("Mejores hiperparámetros:", grid.best_params_)
print("Mejor score CV:", grid.best_score_)

### Regularización: controlando el sobreajuste
La regularización sirve para evitar que el modelo se adapte demasiado a los datos de entrenamiento (overfitting), penalizando coeficientes grandes o complejidad excesiva.
- L1 (Lasso): penaliza la suma de valores absolutos de los coeficientes. Puede eliminar features innecesarias.
- L2 (Ridge): penaliza la suma de los cuadrados de los coeficientes. Reduce los coeficientes grandes, pero no los anula.
- ElasticNet: combina L1 y L2.

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Ridge (L2)
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
print("Ridge RMSE:", round(mean_squared_error(y_test, ridge.predict(X_test), squared=False), 2))

# Lasso (L1)
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
print("Lasso RMSE:", round(mean_squared_error(y_test, lasso.predict(X_test), squared=False), 2))

# ElasticNet (L1 + L2)
en = ElasticNet(alpha=0.1, l1_ratio=0.5)
en.fit(X_train, y_train)
print("ElasticNet RMSE:", round(mean_squared_error(y_test, en.predict(X_test), squared=False), 2))


- alpha controla la fuerza de la penalización: más alto → más regularización.
- Ridge suaviza todos los coeficientes; Lasso puede dejar algunos en cero.
- ElasticNet es útil si queremos un equilibrio entre ambos.

## Paso 11: Evaluación

Una vez elegidas las métricas y evaluados varios modelos, ya tendríamos los resultados del modelo. Sólo queda interpretarlos. Por desgracia este punto va a depender mucho del negocio.

1. Evaluar los resutlados en test: tenemos que comprobar que los resultados no difieren mucho del entrenamiento. Si son inferiores es debido a que el modelo está sobreentrenado (ver abajo para solucionarlo). Si son superiores es raro, probablemente porque no tengamos muchos datos. Habría que intentar coseguir más observaciones.
2. Evaluar los resultados respecto a las necesidades de negocio
3. Almacenar todas las muestras utilizadas en el entrenamiento, así como los scripts y el propio modelo entrenado. La puesta en producción no es el objetivo a cobrir en este notebook.
4. Elegir y evaluar la/las métrica/s con las que presentaremos los resultados del modelo. No tenemos por qué elegir la misma métrica utilizada en la búsqueda del mejor modelo, ya que no siempre es la más entendible: MSE o AUC.

**¿Qué hacer si tenemos overfitting?**

Posibles opciones para reducir el overfitting

- Cross validation: si hemos hecho bien el paso anterior, el cross validation es la mejor técnica para evitar el overfitting.
- Entrenar con más datos: no quitar datos a test, sino intentar conseguir más datos.
- Eliminar features: PCA o un algoritmo de feature selection podría ser una buena solución
- Max depth: reducirlo si estamos con árboles
- Regularización: aumentarla en los modelos que permitan regulaización como regresiones lineales y SVM
- Ensembles: sobretodo los algoritmos de bagging como el random forest no suelen sobreajustarse tanto.

Ejemplo de regresión:

In [ ]:
# Predicciones
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluación
print("Train RMSE:", round(mean_squared_error(y_train, y_train_pred, squared=False), 2))
print("Test RMSE:", round(mean_squared_error(y_test, y_test_pred, squared=False), 2))
print("Train R2:", round(r2_score(y_train, y_train_pred), 2))
print("Test R2:", round(r2_score(y_test, y_test_pred), 2))

# Cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print("Cross-validated R2:", round(cv_scores.mean(), 2))

Si Train RMSE mucho menor que Test RMSE → overfitting

Si Train R2 mucho mayor que Test R2 → overfitting

Cross-validation nos da una idea más robusta del desempeño general del modelo.

Ejemplo de clasificación:

In [ ]:
# Predicciones
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Métricas
print("Train Accuracy:", round(accuracy_score(y_train, y_train_pred), 3))
print("Test Accuracy:", round(accuracy_score(y_test, y_test_pred), 3))
print("Test Precision:", round(precision_score(y_test, y_test_pred), 3))
print("Test Recall:", round(recall_score(y_test, y_test_pred), 3))
print("Test F1:", round(f1_score(y_test, y_test_pred), 3))

# Matriz de confusión
c_matrix = confusion_matrix(y_test, y_test_pred)
sns.heatmap(c_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Cross-validation para detectar overfitting
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Cross-validated Accuracy:", round(cv_scores.mean(), 3))

Interpretación:
- Si Train Accuracy mucho mayor que Test Accuracy → overfitting
- Precision y Recall ayudan a entender errores según la clase:
  - Recall alto → pocos falsos negativos
  - Precision alto → pocos falsos positivos
- La matriz de confusión muestra en detalle dónde el modelo se confunde.
- Cross-validation da una medida más estable y general del desempeño del modelo.

## Paso 12: Conclusión

Al finalizar un proyecto de ML, las conclusiones deben centrarse en estos puntos:
1. Rendimiento general del modelo
- Qué tan bien predice en comparación con los datos de test.
- Métricas clave (accuracy, F1, R², MSE, etc.) y su interpretación concreta: “El modelo acierta un 82% de los casos y su F1-score indica un buen equilibrio entre precisión y recall”.
2. Errores y limitaciones
- Identificar patrones de error: qué casos falla más y por qué.
- Posibles sesgos: datos desbalanceados, features que afectan demasiado o poco.
- Riesgos de overfitting o underfitting.
3. Importancia de variables / interpretabilidad
- Qué features fueron más determinantes en las predicciones.
- Cómo esas variables se relacionan con el resultado esperado.
4. Aplicabilidad para el negocio o contexto
- Cómo los resultados ayudan a la toma de decisiones.
- Limitaciones prácticas: casos donde no conviene usar el modelo o donde los errores pueden ser críticos.
5. Recomendaciones futuras
- Mejoras posibles: más datos, refinamiento de features, ajuste de hiperparámetros, pruebas con otros modelos.
- Estrategias para mantener el modelo actualizado y confiable en producción.

**Ejemplo de redacción para presentar resultados:**

*“El modelo de clasificación logra un 82% de accuracy y un F1-score de 0.77, lo que indica un buen equilibrio entre precisión y recall. Detecta correctamente la mayoría de los casos positivos, aunque confunde principalmente X con Y. Las variables más relevantes fueron A, B y C, lo que concuerda con el conocimiento del negocio. Se recomienda seguir recopilando datos y evaluar regularmente el rendimiento para evitar sesgos futuros.”*